In [ ]:
from bayesbeat.result import get_fit
from bayesbeat.data import get_data, get_n_entries
import h5py
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pathlib

from utils import (
    get_duration,
    add_log10_bayes_factor_colorbar,
    get_bayes_factor_markers,
    get_frequency,
)

plt.style.use("paper.mplstyle")

In [ ]:
outdir = pathlib.Path("figures")
outdir.mkdir(exist_ok=True)

file_format = "pdf"

Path to the data file

In [ ]:
data_file = "../data/PyTotalAnalysis_2024_02_23.mat"
n_ringdowns = get_n_entries(data_file)

In [ ]:
paths = {
    "simple_model_amp_noise": pathlib.Path("../analysis/gens_data/rundir/simple_model_both_noise_sources_fix/"),
    "terms_1": pathlib.Path("../analysis/gens_data/rundir/analytic_model_1_term_both_noise_sources/"),
    "terms_3": pathlib.Path("../analysis/gens_data/rundir/analytic_model_3_terms_both_noise_sources/"),
    "terms_5": pathlib.Path("../analysis/gens_data/rundir/analytic_model_5_terms_both_noise_sources/"),
    "terms_7": pathlib.Path("../analysis/gens_data/rundir/analytic_model_7_terms_both_noise_sources/"),
    "terms_1_no_gap": pathlib.Path("../analysis/gens_data/rundir/analytic_model_1_term_both_noise_sources_no_gap/"), 
    "terms_3_no_gap": pathlib.Path("../analysis/gens_data/rundir/analytic_model_3_terms_both_noise_sources_no_gap/"),
    "terms_5_no_gap": pathlib.Path("../analysis/gens_data/rundir/analytic_model_5_terms_both_noise_sources_no_gap/"),
    "terms_7_no_gap": pathlib.Path("../analysis/gens_data/rundir/analytic_model_7_terms_both_noise_sources_no_gap/"),
}

In [ ]:
log_z = {}
for key, path in paths.items():
    log_z[key] = np.empty(n_ringdowns)
    for index in range(n_ringdowns):
        result_file=path / "analysis" / f"index_{index}" / "result.hdf5"
        try:
            with h5py.File(result_file, "r") as res_file:
                log_z[key][index] = res_file["log_evidence"][()]
        except OSError:
            log_z[key][index] = np.nan

In [ ]:
parameters = {}
for key, path in paths.items():
    d = dict(
        a_1=np.nan * np.empty(n_ringdowns),
        a_ratio=np.nan * np.empty(n_ringdowns),
        a_scale=np.nan * np.empty(n_ringdowns),
    )
    for index in range(n_ringdowns):
        result_file = path / "analysis" / f"index_{index}" / "result.hdf5"
        try:
            with h5py.File(result_file, "r") as res_file:
                posterior_samples = res_file["posterior_samples"][()]
            max_logl_idx = np.argmax(posterior_samples["logL"])
            for p in posterior_samples.dtype.names:
                if p not in d:
                    d[p] = np.empty(n_ringdowns)
                d[p][index] = posterior_samples[p][max_logl_idx]
        except FileNotFoundError:
            pass
    parameters[key] = d

In [ ]:
durations = np.empty(n_ringdowns)
for index in range(n_ringdowns):
    durations[index] = get_duration(data_file, index)

In [ ]:
freqs = np.empty(n_ringdowns)
for index in range(n_ringdowns):
    freqs[index] = get_frequency(data_file, index)

## Bayes Factors

### Comparison to simple model

In [ ]:
fig, axs = plt.subplots()

model_1 = "terms_3"
model_2 = "simple_model_amp_noise"
x_values = (parameters[model_1]["a_1"]) #* parameters["simple_model_amp_noise"]["a_1"])
y_values = parameters[model_1]["a_ratio"] * parameters[model_1]["a_1"]
log10_bf = (log_z[model_1] - log_z[model_2]) / np.log(10)


signs = np.sign(log10_bf)
cvar = log10_bf#signs * np.log10(np.abs(log10_bf))
cvar_max = np.nanmax(log10_bf)#np.round(np.nanmax(np.abs(cvar)))
threshold = 10
threshold_colour = "k"
markers = get_bayes_factor_markers(cvar, threshold)

for (i, v), marker in zip(enumerate(cvar), markers):
    simple_scatter = axs.scatter(
        x_values[i],
        y_values[i],
        s=20,
        c=v,
        cmap="RdYlBu",
        vmin=-cvar_max,
        vmax=cvar_max,
        marker=marker,
        edgecolor="k",
        linewidths=0.5,
    )

# add_log10_bayes_factor_colorbar(simple_scatter, ax=axs)
cbar = plt.colorbar(simple_scatter, ax=axs, label=r"$\log_{10} \cal{B}$", extend="both")
cbar.ax.axhline(threshold, c=threshold_colour)
cbar.ax.axhline(-threshold, c=threshold_colour)

axs.set_xscale("log")
axs.set_yscale("log")
xlim = axs.get_xlim()
ylim = axs.get_ylim()

axs.fill_between([1e-7, 2e-3], [1e-7, 2e-3], [2e-3, 2e-3], color="lightgrey", zorder=0)

# axs.legend(handles=legend_handles, frameon=True, ncol=3)
axs.set_xlabel(r"$a_1$")
axs.set_ylabel(r"$a_2$")
axs.set_xlim(xlim)
axs.set_ylim(ylim)
fig.savefig(outdir / f"analytic_bf_simple.{file_format}")

In [ ]:
fig, axs = plt.subplots()

model_1 = "terms_1"
model_2 = "simple_model_amp_noise"
x_values = (parameters[model_1]["a_1"]) #* parameters["simple_model_amp_noise"]["a_1"])
y_values = parameters[model_1]["a_ratio"] * parameters[model_1]["a_1"]
log10_bf = (log_z[model_1] - log_z[model_2]) / np.log(10)

signs = np.sign(log10_bf)
cvar = log10_bf#signs * np.log10(np.abs(log10_bf))
cvar_max = np.nanmax(log10_bf)#np.round(np.nanmax(np.abs(cvar)))
threshold = 10
threshold_colour = "k"
markers = get_bayes_factor_markers(cvar, threshold)

for (i, v), marker in zip(enumerate(cvar), markers):
    simple_scatter = axs.scatter(
        x_values[i],
        y_values[i],
        s=20,
        c=v,
        cmap="RdYlBu",
        vmin=-cvar_max,
        vmax=cvar_max,
        marker=marker,
        edgecolor="k",
        linewidths=0.5,
    )

# add_log10_bayes_factor_colorbar(simple_scatter, ax=axs)
cbar = plt.colorbar(simple_scatter, ax=axs, label=r"$\log_{10} \cal{B}$", extend="both")
cbar.ax.axhline(threshold, c=threshold_colour)
cbar.ax.axhline(-threshold, c=threshold_colour)

axs.set_xscale("log")
axs.set_yscale("log")
xlim = axs.get_xlim()
ylim = axs.get_ylim()
axs.fill_between([1e-6, 1e-2], [1e-6, 1e-2], [1e-1, 1e-1], color="lightgrey", zorder=0)

# axs.legend(handles=legend_handles, frameon=True, ncol=3)
axs.set_xlabel(r"$a_1$")
axs.set_ylabel(r"$a_2$")
axs.set_xlim(xlim)
axs.set_ylim(ylim)
axs.set_xlim(0.92e-3)
fig.savefig(outdir / f"analytic_bf_simple_1_term.{file_format}")

In [ ]:
1.1 * 1e-6

### Number of terms

In [ ]:
baseline = "terms_1"
include = ["terms_3"]

In [ ]:
labels = {
    "terms_1" : "1/5",
    "terms_3" : "3/1",
    "terms_5" : "5/1",
    "terms_7" : "7/1",
}

In [ ]:
parameters[model_1].keys()

In [ ]:
pairs = [
    ("terms_3", "terms_1"),
    ("terms_5", "terms_3"),
    ("terms_7", "terms_5"),
]
labels = ["3/1", "5/3", "7/5"]
thresholds = [2, 2, 2]

In [ ]:
for (j, (model_1, model_2)), threshold in zip(enumerate(pairs), thresholds):

    fig, axs = plt.subplots()
    # axs.grid()
    x_values = (parameters[model_1]["a_1"]) #* parameters["simple_model_amp_noise"]["a_1"])
    y_values = parameters[model_1]["a_ratio"] * parameters[model_1]["a_1"]
    log10_bf = (log_z[model_1] - log_z[model_2]) / np.log(10)
    print(np.where(np.abs(log10_bf) > threshold ))
    # threshold = 1
    
    # log10_bf = (log_z[model_1] - log_z[model_2]) / np.log(10)
    signs = np.sign(log10_bf)
    cvar = log10_bf#signs * np.log10(np.abs(log10_bf))
    cvar_max = np.nanmax(log10_bf)#np.nanmax(np.abs(cvar)))
    markers = get_bayes_factor_markers(cvar, threshold=threshold)
    
    for (i, v), marker in zip(enumerate(cvar), markers):
        simple_scatter = axs.scatter(
            x_values[i],
            y_values[i],
            s=20,
            c=v,
            cmap="RdYlBu",
            vmin=-cvar_max,
            vmax=cvar_max,
            marker=marker,
            edgecolor="k",
            linewidths=0.5
        )
        # axs.text(x_values[i], y_values[i], str(i), fontsize=6)
    
    axs.set_xscale("log")
    axs.set_yscale("log")
    xlim = axs.get_xlim()
    ylim = axs.get_ylim()
    
    axs.fill_between([1e-7, 1e-2], [1e-7, 1e-2], [1e-2, 1e-2], color="lightgrey", zorder=0)
    
    cbar = plt.colorbar(simple_scatter, ax=axs, label=r"$\log_{10} \mathcal{B}$"  , extend="both")
    cbar.ax.axhline(threshold, c=threshold_colour)
    cbar.ax.axhline(-threshold, c=threshold_colour)
    
    axs.set_xlabel(r"$a_1$")
    axs.set_ylabel(r"$a_2$")
    
    axs.set_xlim(xlim)
    axs.set_ylim(ylim)
    # fig.suptitle(f"{model_1} - {model_2}")
    plt.tight_layout()
    fig.savefig(outdir / f"analytic_bf_{model_1}_{model_2}.{file_format}")


## Without gap

In [ ]:
pairs = [
    ("terms_3", "terms_3_no_gap"),
    ("terms_5", "terms_5_no_gap"),
    ("terms_7", "terms_7_no_gap"),
]
labels = [r"\text{G/NG}", r"\text{G/NG}", r"\text{G/NG}", r"\text{G/NG}"]

In [ ]:
for (j, (model_1, model_2)), threshold in zip(enumerate(pairs), thresholds):

    fig, axs = plt.subplots()
    # axs.grid()
    x_values = (parameters[model_1]["a_1"]) #* parameters["simple_model_amp_noise"]["a_1"])
    y_values = parameters[model_1]["a_ratio"] * parameters[model_1]["a_1"]
    log10_bf = (log_z[model_1] - log_z[model_2]) / np.log(10)
    print(log10_bf)
    # log10_bf = (log_z[model_1] - log_z[model_2]) / np.log(10)
    signs = np.sign(log10_bf)
    cvar = log10_bf#signs * np.log10(np.abs(log10_bf))
    cvar_max = np.nanmax(log10_bf)#np.nanmax(np.abs(cvar)))
    markers = get_bayes_factor_markers(cvar, threshold=threshold)
    
    for (i, v), marker in zip(enumerate(cvar), markers):
        simple_scatter = axs.scatter(
            x_values[i],
            y_values[i],
            s=20,
            c=v,
            cmap="RdYlBu",
            vmin=-cvar_max,
            vmax=cvar_max,
            marker="o",
            edgecolor="k",
            linewidths=0.5
        )
    
    axs.set_xscale("log")
    axs.set_yscale("log")
    xlim = axs.get_xlim()
    ylim = axs.get_ylim()
    
    axs.fill_between([1e-7, 1e-2], [1e-7, 1e-2], [1e-2, 1e-2], color="lightgrey", zorder=0)
    
    cbar = plt.colorbar(simple_scatter, ax=axs, label=r"$\log_{10} \mathcal{B}_{" + f"{labels[j]}" + r"}$"  , extend="both")
    # cbar.ax.axhline(threshold, c=threshold_colour)
    # cbar.ax.axhline(-threshold, c=threshold_colour)
    
    axs.set_xlabel(r"$a_1$")
    axs.set_ylabel(r"$a_2$")
    
    axs.set_xlim(xlim)
    axs.set_ylim(ylim)
    # fig.suptitle(f"{model_1} - {model_2}")
    plt.tight_layout()
    fig.savefig(outdir / f"analytic_bf_{model_1}_{model_2}.{file_format}")
